# Regression with Keras - A

## Introduction

Keras is a high-level API for building deep learning models. It has gained favor for its ease of use and syntactic simplicity facilitating fast development. Building a very complex deep learning network can be achieved with Keras with only few lines of code. In this notebook we perform Regression using Keras on a dataset containing compressive strength of different samples of concrete based on the volumes of the different materials that were used to make them.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3> 
1. <a href="#item1">Download and Clean Dataset</a>  
2. <a href="#item2">Import Keras</a>  
3. <a href="#item3">Build a Neural Network</a>  
4. <a href="#item4">Train and Test the Network</a>
5. <a href="#item4">Listing MSE</a>      
</font>
</div>

<a id="item1"></a>

## Download and Clean Dataset

Let's start by importing the <em>pandas</em> and the Numpy libraries.

In [1]:
#Import required libraries:
import pandas as pd
import numpy as np

<strong>This dataset is about the compressive strength of different samples of concrete based on the volumes of the different materials that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>

Let's download the data and read it into a <em>pandas</em> dataframe.

In [2]:
concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 

#### Let's check how many data points we have.

In [3]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.

Let's check the dataset for any missing values.

In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.

#### Split data into predictors and target

In [6]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

<a id="item2"></a>

Let's do a quick sanity check of the predictors and the target dataframes.

In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [9]:
n_cols = predictors.shape[1] # number of predictors

<a id="item1"></a>

## Import Keras

Keras normally runs on top of a low-level library such as TensorFlow. This means that to be able to use the Keras library, you will have to install TensorFlow first and when you import the Keras library, it will be explicitly displayed what backend was used to install the Keras library.

#### Let's go ahead and import the Keras library

In [10]:
import keras

2023-10-24 21:45:07.943260: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64
2023-10-24 21:45:07.943300: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


As you can see, the TensorFlow backend was used to install the Keras library.

Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.

In [11]:
from keras.models import Sequential
from keras.layers import Dense

## Build a Neural Network

Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.

In [12]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function creates a model that has one hidden layer with 10 neurons and a ReLU activation function. It uses the adam optimizer and the mean squared error as the loss function.

Let's import train_test_split function from scikit-learn in order to split the data into a training and test sets.

In [13]:
from sklearn.model_selection import train_test_split

Splitting the data by reserving 30% for test set.

In [15]:
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=42)

## Train and Test the Network

In [17]:
# build the model
model = regression_model()

# fit the model
model.fit(predictors, target, epochs=50, verbose=2)

Epoch 1/50
33/33 - 1s - loss: 3986.9353 - 1s/epoch - 43ms/step
Epoch 2/50
33/33 - 0s - loss: 1271.3153 - 46ms/epoch - 1ms/step
Epoch 3/50
33/33 - 0s - loss: 930.1856 - 60ms/epoch - 2ms/step
Epoch 4/50
33/33 - 0s - loss: 778.4827 - 64ms/epoch - 2ms/step
Epoch 5/50
33/33 - 0s - loss: 696.5660 - 51ms/epoch - 2ms/step
Epoch 6/50
33/33 - 0s - loss: 614.2612 - 55ms/epoch - 2ms/step
Epoch 7/50
33/33 - 0s - loss: 540.6136 - 55ms/epoch - 2ms/step
Epoch 8/50
33/33 - 0s - loss: 477.0742 - 57ms/epoch - 2ms/step
Epoch 9/50
33/33 - 0s - loss: 409.8828 - 54ms/epoch - 2ms/step
Epoch 10/50
33/33 - 0s - loss: 359.9341 - 54ms/epoch - 2ms/step
Epoch 11/50
33/33 - 0s - loss: 309.4772 - 55ms/epoch - 2ms/step
Epoch 12/50
33/33 - 0s - loss: 269.1745 - 61ms/epoch - 2ms/step
Epoch 13/50
33/33 - 0s - loss: 239.7222 - 47ms/epoch - 1ms/step
Epoch 14/50
33/33 - 0s - loss: 214.4732 - 57ms/epoch - 2ms/step
Epoch 15/50
33/33 - 0s - loss: 193.6304 - 54ms/epoch - 2ms/step
Epoch 16/50
33/33 - 0s - loss: 178.7528 - 62ms/e

We need to evaluate the model on test data.


In [18]:
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

10/10 [==============================] - 0s 2ms/step


103.99992370605469

Now we need to compute the mean squared error between the predicted concrete strength and the actual concrete strength.

Let's import the mean_squared_error function from Scikit-learn.

In [19]:
from sklearn.metrics import mean_squared_error
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

103.99992231759127 0.0


List of 50 mean squared errors with report of mean and the standard deviation of the mean squared errors.

In [20]:
total_mean_squared_errors = 50
epochs = 50
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Mean and standard deviation of " +str(total_mean_squared_errors) + " mean squared errors without normalized data. \n Total number of epochs for each training is: " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 96.52924346923828
10/10 [==============================] - 0s 1ms/step
MSE 2: 117.88529205322266
10/10 [==============================] - 0s 2ms/step
MSE 3: 101.39268493652344
10/10 [==============================] - 0s 2ms/step
MSE 4: 67.71676635742188
10/10 [==============================] - 0s 2ms/step
MSE 5: 49.27994918823242
10/10 [==============================] - 0s 2ms/step
MSE 6: 59.182090759277344
10/10 [==============================] - 0s 1ms/step
MSE 7: 58.394466400146484
10/10 [==============================] - 0s 2ms/step
MSE 8: 47.8305549621582
10/10 [==============================] - 0s 1ms/step
MSE 9: 49.44721984863281
10/10 [==============================] - 0s 1ms/step
MSE 10: 52.161434173583984
10/10 [==============================] - 0s 1ms/step
MSE 11: 48.84428787231445
10/10 [==============================] - 0s 1ms/step
MSE 12: 49.170833587646484
10/10 [==============================] - 0s 1ms/step
MSE 13: 62.60831832885742
10/10 [=======================